# Importar librerias

In [23]:
import numpy as np

import matplotlib.pyplot as plt
import sklearn
%load_ext cudf.pandas
# pandas API is now GPU accelerated

import pandas as pd

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas


# Importar datos

In [24]:
#Cargamos los datos
! pip install openpyxl
data = pd.read_excel('data/bosch_aic_datathon.xlsx')

# Hacemos copia de los datos
data_sucia = data
data_sucia.head()

,Posición,Area,Á.liq.,Grupo de personal,CODIGO POSTAL,Motivo de la RENUNCIA,Banda,Baja,ReglaPHT,Alta,Años,Meses,Antigüedad,Clave de sexo,Lugar de nacimiento,Nacionalidad,Edad del empleado,¿Cuanto tiempo tiene viviendo en Cd. Juarez?,Estado Civil,Hijos
0,POS514,LOG,S6,Exempt,32520,Vol-Otro empleo,E3,2023-01-02,ADMINIST,2018-06-21,5,55.200000,1656,Femenino,JUAREZ,Mexican,31,None,solter,0
1,POS1745,MFS,S5,Direct,32217,Vol-Otro empleo,DNO,2023-01-03,ADMINIST,2017-07-20,6,66.433333,1993,Masculino,DURANGO,Mexican,25,10+ Years,solter,0
2,POS1230,VS7,S5,Direct,32600,Vol-Otro empleo,DNO,2023-01-03,JT012324,2019-08-29,4,40.766667,1223,Masculino,JUAREZ,Mexican,23,10+ Years,solter,0
3,POS1661,MFS,S5,Direct,32575,Vol-Estudios,DO,2023-01-03,JT012324,2022-11-03,1,2.033333,61,Femenino,CIUDAD JUAREZ,Mexican,18,10+ Years,solter,0
4,POS1921,MFS,S5,Direct,32575,Vol-Cambio de domicilio,DO,2023-01-03,JT012324,2017-03-02,6,71.100000,2133,Masculino,JUAREZ,Mexican,26,10+ Years,solter,0


# Limpieza por columnas

### Posicion

### Area

### A Liquida

### A Liquida

### A Liquida

### A Liquida

### Hijos - Victor

### Lugar de nacimiento

In [25]:
# Limpia los datos de la columna Lugar de nacimiento a el nombre del estado de nacimiento. 
# Limpia los datos de la columna Lugar de nacimiento a el nombre del estado de nacimiento. 
# Diccionario de reemplazos
reemplazos = {
    'Chihuahua': ['AZTECA CIUDAD JUAREZ', 'CAMARGO', 'CD JUAREZ', 'CD JUAREZ CHIH', 'CD. JUAREZ'],
    'Durango': ['CD. GOMEZ PALACIO', 'GENERAL SIMON BOLIVAR', 'GOMEZ PALACIO', 'LERDO', 'DURANGO', 'PEÑON BLANCO', 'POBLADO EL LUCERO', 'SAN DIMAS', 'TLAHUALILO DE ZARAGOZA'],
    'Aguascalientes': ['AGUASCALIENTES'],
    'Sinaloa': ['AHOME', 'CULIACAN'],
    'Veracruz': ['ANGEL R CABADA', 'COATZACOALCOS', 'CORDOBA', 'CORDOVA', 'COSAMALOAPAN', 'LAS MERCEDES', 'MARTINEZ DE LA TORRE', 'MINATITLAN', 'MOLOACAN', 'RAYA CARACOL', 'SAN ANDRES TUXTLA', 'TRES VALLES', 'VERACRUZ'],
    # Agrega el resto de tus reemplazos aquí
}

# Invertir el diccionario para mapear cada ciudad a su estado correspondiente
reemplazos_invertidos = {ciudad: estado for estado, ciudades in reemplazos.items() for ciudad in ciudades}

# Realizar los reemplazos
data_sucia['Lugar de nacimiento'] = data_sucia['Lugar de nacimiento'].replace(reemplazos_invertidos)

# Ver resultados de los reemplazos usando aleatoriedad
data_sucia['Lugar de nacimiento'].sample(10)

# Diccionario para la nueva celda de reemplazos
reemplazos_nueva_celda = {
    'PUEBLA': ['GUADALUPE ANALCO', 'PUEBLA'],
    'OAXACA': ['IXHUATAN', 'OAXACA', 'PASO DEL JOBO', 'SAN FRANCISCO DEL MAR', 'SAN JUAN BAUTISTA TUXTEPE', 'SAN LUCAS OJITLAN', 'TUXTEPEC'],
    'TAMAULIPAS': ['MATAMOROS']
}

# Invertir el diccionario para mapear cada ciudad a su estado correspondiente
reemplazos_invertidos_nueva_celda = {ciudad: estado for estado, ciudades in reemplazos_nueva_celda.items() for ciudad in ciudades}

# Realizar los reemplazos
data_sucia['Lugar de nacimiento'] = data_sucia['Lugar de nacimiento'].replace(reemplazos_invertidos_nueva_celda)

data_sucia['Lugar de nacimiento'].sample(10)


399            CHIHUAHUA
452            CHIHUAHUA
723            MICHOACAN
274            Chihuahua
868            Chihuahua
46      CUAUHTEMOC, CHIH
25                   NaN
298               JUAREZ
819                TEXAS
1072           CHIHUAHUA
Name: Lugar de nacimiento, dtype: object

[ ] Posición: No tenemos información. -
[ ] Área: Luis pasará un mensaje en WhatsApp explicando el significado de las iniciales.
[ ] A líquida: No disponible.
[ ] Grupo de personal: Luis proporcionará detalles en WhatsApp.
[ ] Código Postal: Usar API para determinar el lugar actual de vivienda. Dos códigos no corresponden.
[ ] Motivo de la Renuncia: Continuar con la limpieza y clasificación de las razones, diferenciando entre renuncias voluntarias e impuestas.
[ ] Baja: Fecha en que los empleados salen de la empresa.
[ ] Alta: Fecha en que los empleados entran a la empresa.
[ ] Regla PHT: Se desconoce su significado.
[ ] Años: Duración en años que el empleado estuvo en la empresa.
[ ] Meses: Duración en meses que el empleado estuvo en la empresa.
[ ] Antigüedad: Total de días que el empleado estuvo en la empresa.
[ ] Clave de sexo: M (Masculino) y F (Femenino).
[ ] Lugar de Nacimiento: Ya está limpio. Falta convertir a mayúsculas.
[ ] Nacionalidad: Información disponible.
[ ] Edad del Empleado: Información disponible.
[ ] Tiempo Viviendo en Ciudad Juárez: Convertir a categoría.
[ ] Estado Civil: Incluye soltero, casado, separado.
[ ] Hijos: Número de hijos. Crear nueva columna para indicar si tienen hijos o no.

### Edad del empleado

In [26]:
# Edad del empleado
# Convertir en tipos de datos enteros
data_sucia['Edad del empleado'] = data_sucia['Edad del empleado'].astype(int)
# Cambiar nombre de columna a Edad
data_sucia = data_sucia.rename(columns={'Edad del empleado': 'Edad'})

# Ver resultados de los reemplazos usando aleatoriedad